In [23]:
import pandas as pd
import numpy as np
import pygsheets
#authorization
gc = pygsheets.authorize(service_file='/Users/vitale/Sites/project1/viz-midterm-56fb8f946d76.json')
sh = gc.open('viz midterm data')

missing_countries_list = []


In [3]:
# some useful sheets api code
# Loop through each variable and create a new sheet of that variable per year per country in google sheets
# plot_vars = [""]
# for var in total_df_pivot['variable'].unique():
#     df = total_df_pivot[total_df_pivot.variable == var].drop(columns="variable").reset_index().drop(columns="index")
#     sh.add_worksheet(var)
#     wks = sh.worksheet_by_title(var)
#     wks.set_dataframe(df,(1,1))
#     print(var)
    
#delete worksheets when necessary
# for i in range(6):
#     sh.del_worksheet(sh[i])

In [4]:
# parse military data

xldata = pd.ExcelFile("https://www.sipri.org/sites/default/files/SIPRI-Milex-data-1949-2017.xlsx")

# Data for military expenditure by country in current US$ (millions), presented according to calendar year.
me_total_usd_main = pd.read_excel(xldata,sheet_name="Current USD",skiprows=5)
# Data for military expenditure by country as a share of GDP, presented according to calendar year.
me_share_gdp_main = pd.read_excel(xldata,sheet_name="Share of GDP",skiprows=5)
# Data for military expenditure per capita, in current US$, presented according to calender year. (1988-2017 only)
me_percap_main = pd.read_excel(xldata,sheet_name="Per capita",skiprows=6)

#me_percap["Country"].loc(["USA","USSR"])
country_list = ["Argentina", "Australia", "Brazil", "Canada", "China", 
                "France", "Germany", "India", "Indonesia", "Italy", "Japan", 
                "Republic of Korea", "Mexico", "Russia", "Saudi Arabia", 
                "South Africa", "Turkey", "UK", "USA"]
#country_list = ["USA", "China", "Russia", "Germany", "UK", "France", "Italy", "Iran", "Israel", "Saudi Arabia"]
year_list = [2010,2011,2012,2013,2014,2015,2016,2017]


# this standardizes the naming conventions for russia and china, which are different between three data sets
me_total_usd_main.loc[me_total_usd_main["Country"].str.contains("Russia",na=False),"Country"] = "Russia"
me_share_gdp_main.loc[me_share_gdp_main["Country"].str.contains("Russia",na=False),"Country"] = "Russia"
me_percap_main.loc[me_percap_main["Country"].str.contains("Russia",na=False),"Country"] = "Russia"
me_total_usd_main.loc[me_total_usd_main["Country"].str.contains("China",na=False),"Country"] = "China"
me_share_gdp_main.loc[me_share_gdp_main["Country"].str.contains("China",na=False),"Country"] = "China"
me_percap_main.loc[me_percap_main["Country"].str.contains("China",na=False),"Country"] = "China"

# pick out year columns from 2010-2017 for relevant countries
me_total_usd = me_total_usd_main[me_total_usd_main["Country"].isin(country_list)][["Country"] + year_list]
me_share_gdp = me_share_gdp_main[me_share_gdp_main["Country"].isin(country_list)][["Country"] + year_list]
me_percap = me_percap_main[me_percap_main["Country"].isin(country_list)][["Country"] + year_list]

# melt to create "year" and "total_ME" columns
me_total_usd = me_total_usd.melt(id_vars="Country",var_name="year" ,value_name="total_ME")
me_share_gdp = me_share_gdp.melt(id_vars="Country",var_name="year" ,value_name="share_of_gdp_ME")
me_percap = me_percap.melt(id_vars="Country",var_name="year" ,value_name="per_capita_ME")

# merge all into one on country and year
me_df = pd.merge(pd.merge(me_total_usd,me_share_gdp, on=["Country","year"]), me_percap, on=["Country","year"])

#calculate gdp, population, per capita gdp, and share_of_gdp_ME
me_df["gdp"] = me_df["total_ME"]/me_df["share_of_gdp_ME"]
me_df["gdp"] = me_df["gdp"].astype(int)
me_df["pop"] = me_df["total_ME"]/me_df["per_capita_ME"]
me_df["pcgdp"] = me_df["gdp"]/me_df["pop"]
me_df["year"] = me_df["year"].astype(str)
me_df["share_of_gdp_ME"] = me_df["share_of_gdp_ME"]*100

print(len(me_df.Country.unique()) == len(country_list), ": ed_df names and country_list are same not length")
print()
print("Here are our G20 Countries: ",me_df.Country.unique())
print()
print("There are/is", len(list(set(country_list).difference([i for i in me_df.Country.unique()]))), "Countries missing")
print()
print(list(set(country_list).difference([i for i in me_df.Country.unique()])),"are/is missing")
missing_countries_list.extend(list(set(country_list).difference([i for i in me_df.Country.unique()])))


False : ed_df names and country_list are same not length

Here are our G20 Countries:  ['South Africa' 'Mexico' 'Canada' 'USA' 'Argentina' 'Brazil' 'China'
 'Japan' 'India' 'Indonesia' 'Australia' 'Russia' 'France' 'Germany'
 'Italy' 'UK' 'Saudi Arabia' 'Turkey']

There are/is 1 Countries missing

['Republic of Korea'] are/is missing


In [5]:
#EDUCATION DATA SOURCE: https://data.oecd.org/eduresource/public-spending-on-education.htm
# COUNTRY CODES DATA SOURCE: https://unstats.un.org/unsd/tradekb/Knowledgebase/50347/Country-Code

# read in education funding data- EDUPUBEXP (public expenditure) in PC_GDP (percent gdp)
ed_data = pd.read_csv("data/education.csv")

# read in country codes and split country code from name into new columns
country_codes = pd.read_csv("data/country_codes.csv",header=None)
country_codes["LOCATION"], country_codes["name"] = country_codes[0].str.split(' ',1).str

# merge ed_df and country_codes to get  full country names from codes
ed_df_names = pd.merge(ed_data,country_codes,on='LOCATION')

# strip extra string from end of country names
ed_df_names['name'] = ed_df_names['name'].str.strip("\xa0")

# rename countries to match original country list (military data)
ed_df_names['name'] = ed_df_names['name'].replace({'United States': 'USA', 'Russian Federation': 'Russia', 'United Kingdom': 'UK', 'Korea, Republic of': 'Republic of Korea'})

#extract only rows of countries in country list and look at unique countries
ed_df_clean = ed_df_names[ed_df_names['name'].isin(country_list)]
print(len(ed_df_clean.name.unique()) == len(country_list), ": ed_df names and country_list are not same length")
print()
print("Here are our G20 Countries: ",ed_df_clean.name.unique())
print()
print("There are", len(list(set(country_list).difference([i for i in ed_df_clean["name"].unique()]))), "Countries missing")
print()
print(list(set(country_list).difference([i for i in ed_df_clean.name.unique()])),"are/is missing")
missing_countries_list.extend(list(set(country_list).difference([i for i in ed_df_clean.name.unique()])))


True : ed_df names and country_list are not same length

Here are our G20 Countries:  ['Australia' 'Canada' 'France' 'Germany' 'Italy' 'Japan'
 'Republic of Korea' 'Mexico' 'UK' 'USA' 'Brazil' 'India' 'Indonesia'
 'Russia' 'South Africa' 'Turkey' 'Argentina' 'China' 'Saudi Arabia']

There are 0 Countries missing

[] are/is missing


In [6]:
# pull out rows with "PRY_NTRY" (primary not tertiary) for subject and years in our year list
ed_df_clean = ed_df_clean[(ed_df_clean["SUBJECT"] == 'PRY_NTRY') & (ed_df_clean["TIME"].isin(year_list))]
ed_df_clean.shape

(133, 10)

In [7]:
important_ed_cols = ["name","TIME","Value"]
ed_df = ed_df_clean[important_ed_cols]
ed_df.columns = ["Country","year","pct_gdp_ed"]
ed_df['year'] = ed_df['year'].astype(str)
ed_df.head()

/Users/vitale/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Country,year,pct_gdp_ed
17,Australia,2010,3.801
18,Australia,2011,3.499
19,Australia,2012,3.302
20,Australia,2013,3.264
21,Australia,2014,3.212


In [8]:
#HEALTHCARE DATA SOURCE: https://data.oecd.org/healthres/health-spending.htm
#read in healthcare data
hc_data = pd.read_csv('data/healthcare.csv')

#merge healthcare data with country_codes to get full country names
hc_df_names = pd.merge(hc_data,country_codes,on='LOCATION')

# strip extra string from end of country names
hc_df_names['name'] = hc_df_names['name'].str.strip("\xa0")

# rename countries to match original country list (military data)
hc_df_names['name'] = hc_df_names['name'].replace({'United States': 'USA', 'Russian Federation': 'Russia', 'United Kingdom': 'UK', 'Korea, Republic of': 'Republic of Korea'})

#extract only rows of countries in country list and look at unique countries
hc_df_clean = hc_df_names[hc_df_names['name'].isin(country_list)]
print(len(hc_df_clean.name.unique()) == len(country_list), ": ed_df names and country_list are not same length")
print()
print("Here are our G20 Countries: ",hc_df_clean.name.unique())
print()
print("There are", len(set(country_list).difference([i for i in hc_df_clean["name"].unique()])), 
      "Countries missing from healthcare data:",set(country_list).difference([i for i in hc_df_clean["name"].unique()]))
print()
print(list(set(country_list).difference([i for i in hc_df_clean.name.unique()])),"are/is missing")
missing_countries_list.extend(list(set(country_list).difference([i for i in hc_df_clean.name.unique()])))



False : ed_df names and country_list are not same length

Here are our G20 Countries:  ['Australia' 'Canada' 'France' 'Germany' 'Italy' 'Japan'
 'Republic of Korea' 'Mexico' 'Turkey' 'UK' 'USA' 'Brazil' 'China' 'India'
 'Indonesia' 'Russia' 'South Africa']

There are 2 Countries missing from healthcare data: {'Saudi Arabia', 'Argentina'}

['Saudi Arabia', 'Argentina'] are/is missing


In [9]:
# pull out rows with "PRY_NTRY" (primary not tertiary) for subject and years in our year list
hc_df_clean = hc_df_clean[(hc_df_clean["SUBJECT"] == 'TOT') 
                          & (hc_df_clean["TIME"].isin(year_list)) 
                          & (hc_df_clean["MEASURE"] == 'PC_GDP')]
print(hc_df_clean.shape)
hc_df_clean.head()

(125, 10)


,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes,0,name
39,AUS,HEALTHEXP,TOT,PC_GDP,A,2010,8.445,D,AUS Australia,Australia
40,AUS,HEALTHEXP,TOT,PC_GDP,A,2011,8.557,D,AUS Australia,Australia
41,AUS,HEALTHEXP,TOT,PC_GDP,A,2012,8.689,D,AUS Australia,Australia
42,AUS,HEALTHEXP,TOT,PC_GDP,A,2013,8.774,D,AUS Australia,Australia
43,AUS,HEALTHEXP,TOT,PC_GDP,A,2014,9.055,D,AUS Australia,Australia


In [10]:
important_hc_cols = ["name","TIME","Value"]
hc_df = hc_df_clean[important_hc_cols]
hc_df.columns = ["Country","year","pct_gdp_hc"]
hc_df['year'] = hc_df['year'].astype(str)
hc_df.head()

/Users/vitale/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Country,year,pct_gdp_hc
39,Australia,2010,8.445
40,Australia,2011,8.557
41,Australia,2012,8.689
42,Australia,2013,8.774
43,Australia,2014,9.055


In [11]:
# Now merge all of the data together to make for easy transfer to google sheets!!!!


total_df = pd.merge(pd.merge(me_df, ed_df, on=['year','Country'], how='outer'), hc_df, on=['year','Country'], how='outer')
total_df = total_df.fillna(0)
# wks = sh[0]
# wks.title= 'all_data'
# wks.set_dataframe(total_df,(1,1))
# total_df.head()

In [12]:
total_df["total_ED"] = (total_df["pct_gdp_ed"] * (total_df["gdp"]))/100
total_df["total_ED"] = total_df["total_ED"].astype(int)
total_df["per_capita_ED"] = (total_df["total_ED"]/(total_df["pop"]))
total_df["total_HC"] = (total_df["pct_gdp_hc"] * (total_df["gdp"]))/100
total_df["total_HC"] = total_df["total_HC"].astype(int)
total_df["per_capita_HC"] = (total_df["total_HC"]/(total_df["pop"]))

total_df = total_df.fillna(0)
total_df = total_df.replace(0,"null")
total_df.head()

,Country,year,total_ME,share_of_gdp_ME,per_capita_ME,gdp,pop,pcgdp,pct_gdp_ed,pct_gdp_hc,total_ED,per_capita_ED,total_HC,per_capita_HC
0,South Africa,2010,4188.17,1.11581,81.1902,375348,51.5847,7276.35,null,7.415,null,null,27832,539.54
1,Mexico,2010,5897.2,0.561035,50.2664,1.05113e+06,117.319,8959.58,3.221,5.975,33856,288.581,62804,535.327
2,Canada,2010,19315.7,1.19715,565.304,1.61347e+06,34.1687,47220.7,3.343,10.556,53938,1578.58,170317,4984.6
3,USA,2010,698180,4.66561,2262.11,1.49644e+07,308.641,48484.7,3.607,16.413,539765,1748.84,2456106,7957.8
4,Argentina,2010,3475.35,0.814878,84.3042,426486,41.2239,10345.6,null,null,null,null,null,null


In [13]:
total_df_pivot = total_df.melt(id_vars=["year","Country"],value_vars=[
    "total_ME","share_of_gdp_ME","per_capita_ME","gdp","pop","pcgdp","pct_gdp_ed",
    "pct_gdp_hc","total_ED", "per_capita_ED", "total_HC", "per_capita_HC"
    ]).set_index(["variable","Country"]).pivot(columns='year').reset_index().set_index("Country").reset_index()
total_df_pivot.columns = ["Country","variable","2010","2011","2012","2013","2014","2015","2016","2017"]
total_df_pivot.head()

,Country,variable,2010,2011,2012,2013,2014,2015,2016,2017
0,Argentina,gdp,426486,530158,581431,613316,567069,634031,545216,624485
1,Australia,gdp,1.24665e+06,1.50318e+06,1.56059e+06,1.50499e+06,1.44747e+06,1.22778e+06,1.26062e+06,1.38051e+06
2,Brazil,gdp,2.20883e+06,2.61615e+06,2.46523e+06,2.47282e+06,2.45516e+06,1.80425e+06,1.79642e+06,2.0768e+06
3,Canada,gdp,1.61347e+06,1.78865e+06,1.82429e+06,1.84262e+06,1.79289e+06,1.55281e+06,1.52903e+06,1.64016e+06
4,China,gdp,6.10065e+06,7.5726e+06,8.56051e+06,9.60729e+06,1.04823e+07,1.10647e+07,1.11898e+07,1.19565e+07


In [14]:
#total Military Expenditure
total_me_df = total_df_pivot[
    total_df_pivot.variable == "total_ME"].drop(columns="variable").reset_index().drop(columns="index")

# drop korea because no data
total_me_df = total_me_df[total_me_df.Country != "Republic of Korea"]

wks = sh[0]
wks.title = "total_ME"
wks.set_dataframe(total_me_df,(1,1))
total_me_df.head()

,Country,2010,2011,2012,2013,2014,2015,2016,2017
0,Argentina,3475.35,4051.93,4563.22,5137.97,4979.44,5482.62,4509.65,5680.71
1,Australia,23217.7,26597.2,26216.6,24825.3,25783.7,24045.6,26382.9,27462.3
2,Brazil,34002.9,36936.2,33987,32874.8,32659.6,24617.7,24224.7,29283.5
3,Canada,19315.7,21393.7,20452.1,18515.7,17853.7,17949.1,18132.3,20567
4,China,115712,137967,157390,179880,200772,214093,216031,228231


In [15]:
total_ed_df = total_df_pivot[
    total_df_pivot.variable == "total_ED"].drop(columns="variable").reset_index().drop(columns="index")

# drop countries with no data
total_ed_df = total_ed_df[(total_ed_df.Country != "China") & 
                          (total_ed_df.Country != "Saudi Arabia") &
                         (total_ed_df.Country != "Republic of Korea")]
#drop columns with no values
total_ed_df = total_ed_df.drop(columns=["2016","2017"])

sh.add_worksheet("total_ED")
wks = sh.worksheet_by_title("total_ED")
wks.set_dataframe(total_ed_df,(1,1))
total_ed_df.head()

,Country,2010,2011,2012,2013,2014,2015
0,Argentina,null,null,22414,23839,21735,null
1,Australia,47385,52596,51530,49122,46492,39068
2,Brazil,92793,109433,102750,102671,98746,73270
3,Canada,53938,57987,59070,58097,55920,49208
5,France,95100,99249,92428,96416,97902,82688


In [16]:
total_hc_df = total_df_pivot[
    total_df_pivot.variable == "total_HC"].drop(columns="variable").reset_index().drop(columns="index")

# remove countries with missing data
total_hc_df = total_hc_df[(total_hc_df.Country != "Argentina") &
                         (total_hc_df.Country != "Republic of Korea") &
                          (total_hc_df.Country != "Saudi Arabia")]
                          
sh.add_worksheet("total_HC")
wks = sh.worksheet_by_title("total_HC")
wks.set_dataframe(total_hc_df,(1,1))
total_hc_df.head()

,Country,2010,2011,2012,2013,2014,2015,2016,2017
1,Australia,105279,128627,135599,132047,131068,114552,116632,126081
2,Brazil,176110,204504,192706,197528,207018,160920,null,null
3,Canada,170317,182978,186807,186712,178751,161134,161083,170757
4,China,270441,347430,410818,468835,522962,597714,null,null
5,France,295889,320448,303562,321349,330376,279871,284423,293509


In [17]:
#percent gdp Military Expenditure
pct_gdp_me_df = total_df_pivot[
    total_df_pivot.variable == "share_of_gdp_ME"].drop(columns="variable").reset_index().drop(columns="index")

# drop korea because no data
pct_gdp_me_df = pct_gdp_me_df[pct_gdp_me_df.Country != "Republic of Korea"]

sh.add_worksheet("pct_gdp_ME")
wks = sh.worksheet_by_title("pct_gdp_ME")
wks.set_dataframe(pct_gdp_me_df,(1,1))
pct_gdp_me_df.head()

,Country,2010,2011,2012,2013,2014,2015,2016,2017
0,Argentina,0.814878,0.764287,0.784825,0.837736,0.878101,0.864723,0.82713,0.909663
1,Australia,1.8624,1.76939,1.67992,1.64953,1.78129,1.95845,2.09285,1.98929
2,Brazil,1.53941,1.41185,1.37866,1.32945,1.33024,1.36443,1.3485,1.41003
3,Canada,1.19715,1.19608,1.1211,1.00486,0.995806,1.15591,1.18587,1.25396
4,China,1.89671,1.82193,1.83856,1.87233,1.91534,1.93492,1.93061,1.90884


In [18]:
pct_gdp_ed_df = total_df_pivot[
    total_df_pivot.variable == "pct_gdp_ed"].drop(columns="variable").reset_index().drop(columns="index")

# drop years with no data
pct_gdp_ed_df = pct_gdp_ed_df.drop(columns=["2016","2017"])
# drop countries with no data
pct_gdp_ed_df = pct_gdp_ed_df[(pct_gdp_ed_df.Country != "China") & 
                              (pct_gdp_ed_df.Country != "Saudi Arabia") &
                              (pct_gdp_ed_df.Country != "Republic of Korea")]

sh.add_worksheet("pct_gdp_ED")
wks = sh.worksheet_by_title("pct_gdp_ED")
wks.set_dataframe(pct_gdp_ed_df,(1,1))
pct_gdp_ed_df.head()

,Country,2010,2011,2012,2013,2014,2015
0,Argentina,null,null,3.855,3.887,3.833,null
1,Australia,3.801,3.499,3.302,3.264,3.212,3.182
2,Brazil,4.201,4.183,4.168,4.152,4.022,4.061
3,Canada,3.343,3.242,3.238,3.153,3.119,3.169
5,France,3.593,3.467,3.447,3.433,3.436,3.398


In [19]:
pct_gdp_hc_df = total_df_pivot[
    total_df_pivot.variable == "pct_gdp_hc"].drop(columns="variable").reset_index().drop(columns="index")

# remove countries with missing data
pct_gdp_hc_df = pct_gdp_hc_df[(pct_gdp_hc_df.Country != "Argentina") &
                         (pct_gdp_hc_df.Country != "Republic of Korea") &
                          (pct_gdp_hc_df.Country != "Saudi Arabia")]

sh.add_worksheet("pct_gdp_HC")
wks = sh.worksheet_by_title("pct_gdp_HC")
wks.set_dataframe(pct_gdp_hc_df,(1,1))
pct_gdp_hc_df.head()

,Country,2010,2011,2012,2013,2014,2015,2016,2017
1,Australia,8.445,8.557,8.689,8.774,9.055,9.33,9.252,9.133
2,Brazil,7.973,7.817,7.817,7.988,8.432,8.919,null,null
3,Canada,10.556,10.23,10.24,10.133,9.97,10.377,10.535,10.411
4,China,4.433,4.588,4.799,4.88,4.989,5.402,null,null
5,France,11.179,11.194,11.321,11.442,11.595,11.501,11.539,11.458


In [ ]:
total_df_pivot['variable'].unique()

In [24]:
#add a new sheet for every country
for country in country_list:
    sh.add_worksheet(country)

#add country-specific data to new sheets
for country in country_list:
    wks = sh.worksheet_by_title(country)
    new_frame = total_df[total_df["Country"]==country]
    wks.set_dataframe(new_frame,(1,1))